<a href="https://colab.research.google.com/github/karl-gardner/data_plotting/blob/main/3d_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/karl-gardner/data_plotting  # clone repo
%cd /content/data_plotting

import pandas as pd
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly.graph_objects as go
from google.colab import files
import funcs

Cloning into 'data_plotting'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 30 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.
/content/data_plotting


In [2]:
df = pd.read_excel("/content/drive/MyDrive/nayeem_collaboration/3d_data.xlsx")
pd.set_option('display.max_columns', None)
data = df.drop(columns=["Unnamed: 5", "Unnamed: 11","Unnamed: 17","Unnamed: 18","Unnamed: 23"]).drop(labels=[0,1,13,24,35,36])
data.head()

,Sample 1 (320nm),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Sample 2 (320nm),Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Sample 3 (320nm),Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Sample 4 (130nm),Unnamed: 20,Unnamed: 21,Unnamed: 22,Sample 5 (220nm),Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
2,0.2,1,2.134565,2.022614,2.100857,0.2,1,1.992621,2.20462,2.151456,0.2,1,2.120006,1.594371,2.190793,0.2,1,1.634779,1.57636,0.2,1,2.344948,1.991699,1.968713
3,0.4,1,1.439175,1.390001,1.268413,0.4,1,1.184234,1.180414,1.162054,0.4,1,1.237001,1.197123,1.221109,0.4,1,1.177291,1.261423,0.4,1,1.210531,1.240128,1.252075
4,0.6,1,1.103318,1.085398,1.142127,0.6,1,1.080826,1.086632,1.075917,0.6,1,1.148751,1.105629,1.113484,0.6,1,1.084479,1.098239,0.6,1,1.105082,1.125889,1.113017
5,0.8,1,1.02813,1.039087,1.063873,0.8,1,1.034332,1.028508,1.01758,0.8,1,1.043334,1.052459,1.060375,0.8,1,1.026472,1.036095,0.8,1,1.048111,1.057877,1.048253
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [3]:
# Choose samples to plot

samples = "s12345"



temp,hum,thick,sizes = funcs.compile_data(data,samples)



xdata = np.stack((temp, hum))
X,Y = np.meshgrid(np.arange(np.amin(temp)-0.105, np.amax(temp)+0.12, (np.amax(temp)-np.amin(temp))/100), 
                  np.arange(np.amin(hum)-0.03, np.amax(hum)+0.08, (np.amax(hum)-np.amin(hum))/100))

def func(x, a, b, c, d):
  return a * np.exp(-b * x[0,:]) + c*x[1,:] + d

popt, pcov = optimize.curve_fit(func, xdata, thick)
Z = popt[0]*np.exp(-1*popt[1]*X) + popt[2]*Y + popt[3]


# # This doesn't work for some reason if I do the fig = go.Figure(data=data, layout=layout) way. This must be a bug in plotly
# layout = go.Layout(
#     margin=dict(l=0, r=0, t=0, b=0),
#     width=800*3, height=700*3,
#    yaxis = {"title":{"text":'VALUE'}}
# )


fig = funcs.make_graph(X,Y,Z,temp,hum,thick,sizes,samples)


fig.update_layout(scene_aspectmode='cube', scene = dict(xaxis_title="", yaxis_title="", zaxis_title=""), margin=dict(l=0, r=0, t=0, b=0),font = {"size": 28},
                   width=800, height=700)


config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': '3d_model',
    'height': 700*3,
    'width': 800*3,
    'scale': 5 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.show(config=config)

In [4]:
ssr = np.sum(np.square(thick-func(xdata, *popt)))
sst = np.sum(np.square(thick - np.mean(thick)))
rsquare = 1-ssr/sst

print(f"t={round(popt[0],2)}e^(-{round(popt[1],2)}T)+{round(popt[2],2)}RH+{round(popt[3],2)}")
print("\nR^2: ", round(rsquare,2))
print("\nSSR: ", round(ssr,2))

t=2.07e^(-2.74T)+0.11RH+0.74

R^2:  0.75

SSR:  11.95
